# Testing with Simulated Tracks

In [ ]:
%matplotlib inline

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

import sys
sys.path.append('..')
import pytpc
from pytpc.constants import *
from math import sin, cos
import scipy.signal
from matplotlib import gridspec

In [ ]:
from scipy.interpolate import UnivariateSpline

In [ ]:
sns.set_style('darkgrid')

In [ ]:
tilt = 7*degrees
clock = 6.25  # MHz
emag = 15e3
bmag = 0#0.5691
ef = np.array([0, 0, emag])
bfield_orig = np.array([0, 0, bmag])
trans = np.array([[1, 0, 0],
                  [0, cos(-tilt), -sin(-tilt)],
                  [0, sin(-tilt), cos(-tilt)]])
bf = np.dot(trans, bfield_orig)
vd = pytpc.simulation.drift_velocity_vector(-2, emag, bmag, tilt); vd

In [ ]:
eumat = pytpc.utilities.euler_matrix(pi/2, pi/4, -pi/2); np.around(eumat, decimals=4)

In [ ]:
np.linalg.det(eumat)

In [ ]:
# proj = pytpc.Particle(4, 2, 1, position=(0, 0, 1.), azimuth=0, polar=pi+tilt)
proj = pytpc.Particle(4, 2, 1.5, position=(0, 0, 0.7), azimuth=90*degrees, polar=90*degrees)
target = pytpc.Particle(4, 2, 0)
# heco = pytpc.gases.HeCO2Gas(150.)
heco = pytpc.gases.InterpolatedGasMixture(200., ('helium', 0.9), ('carbon_dioxide', 0.1))

simres = pytpc.simulation.simulate_elastic_scattering_track(proj, target, heco, ef, bf, 5.7, 100*degrees, 90*degrees)
# simres = pytpc.simulation.track(proj, heco, ef, bf)
simres.x *= 1000
simres.y *= 1000
simres.z *= 1000
# simres['pos'][:, 2] = 1000 - simres['pos'][:, 2]

In [ ]:
plt.plot(simres.x, simres.y, '.')
plt.xlim(-275, 275)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')
plt.xlabel('x'); plt.ylabel('y')

In [ ]:
plt.plot(simres.z, simres.y, '.')
plt.xlim(0, 1000)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')
plt.xlabel('z'); plt.ylabel('y')

In [ ]:
plt.plot(simres.z, simres.x, '.')
plt.xlim(0, 1000)
plt.ylim(-275, 275)
plt.gca().set_aspect('equal')
plt.xlabel('z'); plt.ylabel('x')

In [ ]:
evt = pytpc.evtdata.make_event(simres[['x', 'y', 'z']].values, simres.de.values, clock, vd, 
                               40.2, proj.mass_num, 334., 100)

In [ ]:
plt.plot(evt.traces['data'].sum(0))

In [ ]:
pp = pytpc.pad_plot(evt.hits());
# pp.savefig('/Users/josh/Dropbox/fit_pads.pdf', dpi=200, transparent=True)

In [ ]:
xyz = evt.xyzs()
plt.plot(xyz[:, 0], xyz[:, 1], '.')
plt.xlim(-270, 270)
plt.ylim(-270, 270)

In [ ]:
plt.plot(evt.traces['data'].sum(0), 'k--')
for x in evt.traces['data']:
    plt.plot(x)
# plt.savefig('/Users/josh/Dropbox/fit_data.png', dpi=200)

In [ ]:
chfig = pytpc.chamber_plot(evt.xyzs(vd, clock), zscale='dist')
chfig.axes[0].view_init(elev=10, azim=50)
# chfig.savefig('/Users/josh/Dropbox/fit_track.png', transparent=True, dpi=200)

In [ ]:
gs = gridspec.GridSpec(2, 1, height_ratios=(1, 4))

with sns.axes_style('ticks'):
    plt.subplot(gs[0])
    plt.plot(simres.time, simres.de)
    sns.despine()
    plt.gca().set_xticklabels([])
#     plt.gca().set_yticklabels([])

plt.subplot(gs[1])
plt.plot(simres.time, simres.en)
plt.semilogy()

In [ ]:
dpos = np.diff(simres.time) * np.array([pytpc.relativity.beta(e, 4*p_mc2) for e in simres.en[1:]*4]) * c_lgt

In [ ]:
plt.plot(simres.en[1:]*4, heco.energy_loss(simres.en[1:] * 4, 4, 2)*dpos)
# plt.plot(simres.en[1:] * 4, np.diff(simres.time))
plt.gca().invert_xaxis()

In [ ]:
plt.plot(4 - np.cumsum(heco.energy_loss(simres.en[1:] * 4, 4, 2)*dpos))
plt.plot(simres.en[1:]*4)

In [ ]:
ens = np.logspace(-4, 3, 100)
plt.plot(ens, heco.energy_loss(ens, 4, 2) / 1000)
plt.semilogx()

In [ ]:
plt.plot(simres.z, simres.de)
plt.gca().invert_xaxis()

In [ ]:
plt.plot(simres.time)

In [ ]:
plt.plot(simres.time, simres.px)
plt.plot(simres.time, simres.py)
plt.plot(simres.time, simres.pz)

In [ ]:
meas = evt.xyzs(vd, clock, peaks_only=True)[:, 0:3] * 1e-3
meas = meas[meas[:, 2].argsort()][::-1]
meas += (np.array([0, 0, 1]) - meas[0])

act = simres[['x', 'y', 'z', 'px', 'py', 'pz']].values
act[:, 0:3] *= 1e-3
act_times = simres.time

In [ ]:
plt.plot(meas)

In [ ]:
plt.hist(np.linalg.norm(np.diff(meas, axis=0), axis=1), bins=10)

In [ ]:
tr_proj = pytpc.Particle(4, 2, 1.5, position=(0, 0, 1.), azimuth=270*degrees, polar=pi-tilt+pi/8)
tr = pytpc.Tracker(tr_proj, heco, ef, bf, tr_proj.state_vector)
tr.kfilter.Q = np.diag((1e-4, 1e-4, 1e-4, 1e-1, 1e-1, 1e-1))**2 # Process
tr.kfilter.R = np.diag([2e-2]*2 + [4e-2]) ** 2 # Measurement
tr.kfilter.P = np.diag([1e-2] * 3 + [1e-0] * 3)**2

res, covar, res_times = tr.track(meas)

In [ ]:
fig = pytpc.tpcplot.state_vector_plots(x_act=act_times, act=act, x_calc=res_times, calc=res, 
                                       x_data=res_times, data=meas, covar=covar);
# fig.savefig('/Users/josh/Dropbox/fit_with_actual.pdf')

In [ ]:
fig = pytpc.tpcplot.state_vector_plots(x_act=act_times, act=act, x_calc=res_times, calc=res, 
                                       x_data=res_times, data=meas, covar=covar);
# fig.savefig('/Users/josh/Dropbox/fit_with_actual.pdf')

In [ ]:
res_ens = np.sqrt(np.sum(res[:, 3:6]**2, axis=1) + tr_proj.mass**2) - 4*p_mc2

In [ ]:
plt.plot(res[:, 2], res_ens)
plt.plot(act[:, 2], simres.en * 4)

In [ ]:
plt.plot(res_times, res_ens)
plt.plot(simres.time, simres.en * 4)

In [ ]:
plt.plot(res[:, 2], res_times)
plt.plot(simres.z*1e-3, simres.time)